In [ ]:
import os

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

!pip install PyPDF2
!pip install textract
import PyPDF2, textract, nltk

!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
# note: the path may be different for others.
os.chdir('/content/drive/MyDrive/DAP Group 6: Fin Scanners/statements')

Mounted at /content/drive


In [ ]:
!ls

logs			monetary20221102a1.pdf	refsum20220921.gdoc
monetary20220126a1.pdf	monetary20221214a1.pdf	refsum20221102.gdoc
monetary20220504a1.pdf	refsum20220126.gdoc	refsum20221214.gdoc
monetary20220615a1.pdf	refsum20220504.gdoc	results
monetary20220727a1.pdf	refsum20220615.gdoc
monetary20220921a1.pdf	refsum20220727.gdoc


let's try to open one of the statements. Trying out on Dec's speech, since Fed would have been extremely careful with their words then, as compared to in Jan

In [ ]:
# reading the pdf (named 12 for the month december)
monetary_12 = open("monetary20221214a1.pdf", "rb")
monetary_12 = PyPDF2.PdfReader(monetary_12)

text = ""

for page in monetary_12.pages:
    text += page.extract_text()
    if "-0-" in page:
      break;

In [ ]:
# how to know where to split the paragraphs?
# use tabs!

In [ ]:
# remove the first few extra texts
trimmed_text = '\n'.join(text.split('\n', 3)[3:text.index("-0-")])

print(trimmed_text)

 Recent indicators point to modest growth in spending and production.  Job gains have 
been robust in recent months, and the unemployment rate has remained low.  Inflation remains 
elevated, reflecting supply and demand imbalances related to the pandemic, higher food and energy prices, and broader price pressures.  
 Russia’s war against Ukraine is causing tremendous human and economic hardship.  The 
war and related e vents are contributing to upward pressure on inflation and are weighing on 
global economic activity.  The Committee is highly attentive to inflation risks.  
 The Committee seeks to achieve maximum employment and inflation at the rate of 
2 percent over the l onger run.  In support of these goals, the Committee decided to raise the 
target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent.  The Committee anticipates that 
ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary 
policy that is sufficiently restrictive

In [ ]:
split_text = trimmed_text.split("\n")
print(split_text)

[' Recent indicators point to modest growth in spending and production.  Job gains have ', 'been robust in recent months, and the unemployment rate has remained low.  Inflation remains ', 'elevated, reflecting supply and demand imbalances related to the pandemic, higher food and energy prices, and broader price pressures.  ', ' Russia’s war against Ukraine is causing tremendous human and economic hardship.  The ', 'war and related e vents are contributing to upward pressure on inflation and are weighing on ', 'global economic activity.  The Committee is highly attentive to inflation risks.  ', ' The Committee seeks to achieve maximum employment and inflation at the rate of ', '2 percent over the l onger run.  In support of these goals, the Committee decided to raise the ', 'target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent.  The Committee anticipates that ', 'ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary ', 'policy 

In [ ]:
split_text_for_model = ""

for i in range(0, len(split_text)):
  if "Voting" in split_text[i]: 
    break;
  if "-2-" not in split_text[i]:
    split_text_for_model += split_text[i]

print(split_text_for_model)

 Recent indicators point to modest growth in spending and production.  Job gains have been robust in recent months, and the unemployment rate has remained low.  Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic, higher food and energy prices, and broader price pressures.   Russia’s war against Ukraine is causing tremendous human and economic hardship.  The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity.  The Committee is highly attentive to inflation risks.   The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run.  In support of these goals, the Committee decided to raise the target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent.  The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary policy that is sufficiently restrictive to return

looks like all paragraphs in "split_text" start with a "\n"


## Getting the final summary for the model

### Summary using the first sentences

In [ ]:
# tryna extract the first sentences
# logic:
  # if the item starts with a tab or capital letter, then consider it an important sentence
  # if a "." exist in the item, print the whole sentence until the "."
  # else, print the item and the next item until the "." in the next item
  # this is assuming that every sentence is at most 2 lines, which is probably the case
  # if we see the word Voting within the first few chars it means we reached the last para and can break

first_n_summary = ""

for i in range(0, len(split_text)):
  
  if "Voting" in split_text[i][0:7]:
    break;

  if split_text[i][0] == " " and "." in split_text[i]:
    first_n_summary += split_text[i].partition('.')[0].lstrip() + ". "
    print(split_text[i].partition('.')[0].lstrip() + "." + "\n");
  elif split_text[i][0].isupper() and "." in split_text[i]:
    first_n_summary += split_text[i].partition('.')[0] + ". "
    print(split_text[i].partition('.')[0] + "." + "\n");
  elif split_text[i][0] == " " or split_text[i][0].isupper():
    first_n_summary += split_text[i].lstrip() + split_text[i+1].partition('.')[0] + ". "
    print(split_text[i].lstrip() + split_text[i+1].partition('.')[0] + "." + "\n");

Recent indicators point to modest growth in spending and production.

Russia’s war against Ukraine is causing tremendous human and economic hardship.

The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run.

In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook.



### Input for splitting by sentence

In [ ]:
# sentences_summary_input = []

# trimmed_text = ' '.join(text.split('\n', 3)[3:text.index("-0-")])
# current_sentence = ""

# for index,ch in enumerate(trimmed_text):
#   if ch == ".":
#     current_sentence = current_sentence.strip()
#     current_sentence = current_sentence.replace("\n", "")
#     sentences_summary_input.append(current_sentence)
#     current_sentence = ""
#   elif ch == "-":
#     continue
#   elif current_sentence == "Voting":
#     break
#   else:
#     current_sentence += ch
    
# print(sentences_summary_input)

### Input for splitting by paragraph

In [ ]:
para_summary_input = []

trimmed_text2 = text.split('\n')[3:]
current_para = ""
for sentence in trimmed_text2:

  # if first sentence is a " ", that means it's a start of a new paragraph!
  # push the current para into the summary and reset current_para
  if sentence[0] == " ":
    current_para = current_para.replace("(more)", "")
    current_para = current_para.strip()
    para_summary_input.append(current_para.strip())
    current_para = sentence
  # if "-2-" is in the paragraph, then we have to split it up because that sentence contains words from 2 different paragraphs
  # so we add the first part of the split and push to para_summary_input
  # make sure the current sentence resets to the second part of the split
  elif "-2-" in sentence:
    temp = sentence.split("-2-")
    current_para += temp[0]
    current_para = current_para.replace("(more)", "")
    current_para = current_para.strip()
    para_summary_input.append(current_para.strip())
    current_para = temp[1]
  elif "Voting" in sentence:
    temp = sentence.split("Voting")
    current_para += temp[0]
    current_para = current_para.replace("(more)", "")
    current_para = current_para.strip()
    para_summary_input.append(current_para.strip())
    break
  else:
    current_para += sentence

para_summary_input = para_summary_input[1:] # use [1:] to remove the stubborn empty string at the front
print(para_summary_input)

['Recent indicators point to modest growth in spending and production.  Job gains have been robust in recent months, and the unemployment rate has remained low.  Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic, higher food and energy prices, and broader price pressures.', 'Russia’s war against Ukraine is causing tremendous human and economic hardship.  The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity.  The Committee is highly attentive to inflation risks.', 'The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run.  In support of these goals, the Committee decided to raise the target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent.  The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary policy that is sufficiently restrictive to ret

In [ ]:
# import re

# ref_summaries = list(df_test['summary'])

# for i in range (3):
#     candidate_summaries = list(df_test['text'].apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:i+1])))
#     print(f"First {i+1} senctences: Scores {calc_rouge_scores(candidate_summaries, ref_summaries)}")

# Trying out models for zero-shot learning

## financial-summarization-pegasus

In [ ]:
# zero shot model/learning pipeline from hugging face (pegasus model)
# install shit

!pip install transformers
from transformers import pipeline
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:00


In [ ]:
# pegasus for financial data model summary
summarizer1 = pipeline(model = "human-centered-summarization/financial-summarization-pegasus")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")


Default model looks so stupid. We'll have to test out which parameters give us the most favourable results

### Testing out parameters for entire text model

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
# model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

# # Tokenize our text
# # If you want to run the code in Tensorflow, please remember to return the particular tensors as simply as using return_tensors = 'tf'
# input_ids = tokenizer(split_text_for_model, return_tensors="pt").input_ids

# # Generate the output (Here, we use beam search but you can also use any other strategy you like)
# output = model.generate(
#     input_ids, 
#     min_length=50,
#     max_length=5000,
#     num_beams=10, 
#     early_stopping=True
# )

# # Finally, we can print the generated summary
# print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
# output = model.generate(
#     input_ids, 
#     min_length=50,
#     max_length=5000,
#     num_beams=50, 
#     early_stopping=True
# )

# print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
# output = model.generate(
#     input_ids, 
#     min_length=50,
#     max_length=5000,
#     num_beams=20, 
#     early_stopping=True
# )

# print(tokenizer.decode(output[0], skip_special_tokens=True))

### Testing out parameters for split-by-sentences


In [ ]:
# test = []
# for para in para_summary_input:
#   input_ids = tokenizer(para, return_tensors="pt").input_ids
#   output = model.generate(
#       input_ids, 
#       min_length=10,
#       max_length=5000,
#       num_beams=2, 
#       early_stopping=True
#   )
#   print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
# test = []
# for para in para_summary_input:
#   input_ids = tokenizer(para, return_tensors="pt").input_ids
#   output = model.generate(
#       input_ids, 
#       min_length=10,
#       max_length=5000,
#       num_beams=4, 
#       early_stopping=True
#   )
#   print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
# test = []
# for para in para_summary_input:
#   input_ids = tokenizer(para, return_tensors="pt").input_ids
#   output = model.generate(
#       input_ids, 
#       min_length=10,
#       max_length=4000,
#       num_beams=4, 
#       early_stopping=True
#   )
#   print(tokenizer.decode(output[0], skip_special_tokens=True))

After trial and error, the best result for the **entire text** seems to be using the pegasus model with min=50, max=5000 and beam = 10 or 50;

For the **by paragraph**, seems like the model with min=10, max=3000 and beam = 4 is better.

### Final choice of parameters for Pegasus (entire text) candidate

In [ ]:
# Tokenize our text
# If you want to run the code in Tensorflow, please remember to return the particular tensors as simply as using return_tensors = 'tf'
input_ids = tokenizer(split_text_for_model, return_tensors="pt").input_ids

# Generate the output (Here, we use beam search but you can also use any other strategy you like)
output = model.generate(
    input_ids, 
    min_length=50,
    max_length=1024,
    num_beams=10, 
    early_stopping=True
)

# Finally, we can print the generated summary
pegasus_candidate1 = tokenizer.decode(output[0], skip_special_tokens=True)
print(pegasus_candidate1)

In [ ]:
input_ids = tokenizer(split_text_for_model, return_tensors="pt").input_ids

output = model.generate(
    input_ids, 
    min_length=50,
    max_length=1024,
    num_beams=50, 
    early_stopping=True
)

pegasus_candidate2 = tokenizer.decode(output[0], skip_special_tokens=True)
print(pegasus_candidate2)

Economic activity has continued to expand, albeit at a slower pace. Job gains have been robust in recent months, and the unemployment rate has remained low The Committee decided to raise the target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent.


### Final choice of parameters for Pegasus (text split by paragraph) candidate

In [ ]:
pegasus_cand3 = []
for para in para_summary_input:
  input_ids = tokenizer(para, return_tensors="pt").input_ids
  output = model.generate(
      input_ids, 
      min_length=10,
      max_length=1024,
      num_beams=4, 
      early_stopping=True
)
  pegasus_cand3.append(peg_)

pegasus_candidate3 = " ".join(pegasus_cand3)
print(pegasus_candidate3)

Economic recovery is gathering pace. Inflation remains elevated, reflecting supply and demand imbalances War against Ukraine is causing tremendous human and economic hardship. The Committee decided to raise the target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent. The Committee will continue to assess the effects of monetary policy on the U.S. economy.


### First-sentence-of-every-paragraph candidate

In [ ]:
baseline_candidate = first_n_summary
print(baseline_candidate)

Recent indicators point to modest growth in spending and production. Russia’s war against Ukraine is causing tremendous human and economic hardship. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. 


### sshleifer candidate

In [ ]:
# sshleifer/distilbart-cnn-12-6 model summary
summarizer2 = pipeline(model = "sshleifer/distilbart-cnn-12-6")
sshleifer_candidate = summarizer2(split_text_for_model)[0]['summary_text']
print(sshleifer_candidate)

 Recent indicators point to modest growth in spending and production . Job gains have been robust in recent months, and unemployment rate has remained low . Inflation remains elevated, reflecting supply and demand imbalances related to pandemic, higher food and energy prices, and broader price pressures . Russia’s war against Ukraine is causing tremendous human and economic hardship .


In [ ]:
# # if we want to use this model we need to manipulate the tokens to take in larger data size
# # nvr mind this model is shit
# # nadzma/finetuned-T5-UK-financial-summarization
# from tokenizers.pre_tokenizers import Whitespace
# from tokenizers import pre_tokenizers
# from tokenizers.pre_tokenizers import Digits
# pre_tokenizer = pre_tokenizers.Sequence([Whitespace(), Digits(individual_digits=True)])
# pre_tokenizer.pre_tokenize_str(split_text_for_model)

# summarizer3 = pipeline(model = "nadzma/finetuned-T5-UK-financial-summarization")
# print(summarizer3(split_text_for_model[:30]))

In [ ]:
# candidate_summaries = []

# for i, text in enumerate(texts):
#     if i % 100 == 0:
#         print(i)
#     candidate = summarizer(text, min_length=5, max_length=20)
#     candidate_summaries.append(candidate[0]['summary_text'])

## Baseline model evaluation based on ROUGE score

In [ ]:
# !pip install rouge_score
# from rouge_score import rouge_scorer
# !pip install datasets
# from datasets import load_metric
# metric = load_metric("rouge")


In [ ]:
# im not sure why rouge isnt working here but i think we need to create 2 arrays then append the summaries and corresponding references respectively 
# use this stackoverflow to fix im jus lazy HAHA :,)
# https://stackoverflow.com/questions/44026832/valueerror-number-of-features-of-the-model-must-match-the-input

# reference = "Inflation is high and the Committee is highly attentive to inflation risks. The Committee is committed to achieving maximum employment and returning inflation to 2 percent. The federal funds rate will be raised to 4-1/4 to 4-1/2 percent. The Committee will continue to reduce its holdings of securities. For future raises in the target range, the Committee will look at the cumulative tightening of monetary policy, the lag time of monetary policy, and economic and financial developments."

# def calc_rouge_scores(candidates, references):
#     result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
#     result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
#     return result

# print(calc_rouge_scores(baseline_candidate, reference))

In [ ]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

def calc_rouge_scores(model_summary, reference_summary):
    return rouge.get_scores(model_summary, reference_summary)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
reference = "Inflation is high and the Committee is highly attentive to inflation risks. The Committee is committed to achieving maximum employment and returning inflation to 2 percent. The federal funds rate will be raised to 4-1/4 to 4-1/2 percent. The Committee will continue to reduce its holdings of securities. For future raises in the target range, the Committee will look at the cumulative tightening of monetary policy, the lag time of monetary policy, and economic and financial developments."

baseline_results = calc_rouge_scores(baseline_candidate, reference)
print(baseline_results)

[{'rouge-1': {'r': 0.4, 'p': 0.38461538461538464, 'f': 0.3921568577470204}, 'rouge-2': {'r': 0.18840579710144928, 'p': 0.203125, 'f': 0.19548871681157795}, 'rouge-l': {'r': 0.34, 'p': 0.3269230769230769, 'f': 0.3333333283352557}}]


# Zero-Shot Learning evaluation based on ROUGE score

Zero-Shot Learning is refers to using pre-trained models to evaluate test data of classes that have not been used during training.
We will test a range of models to see which gives us the highest ROUGE score, and train that model with our data.

In [ ]:
# Testing out pegasus model entire text

pegasus_candidate1_results = calc_rouge_scores(pegasus_candidate1, reference)
print(pegasus_candidate1_results)

[{'rouge-1': {'r': 0.24, 'p': 0.2926829268292683, 'f': 0.2637362587851709}, 'rouge-2': {'r': 0.057971014492753624, 'p': 0.0851063829787234, 'f': 0.06896551242122508}, 'rouge-l': {'r': 0.22, 'p': 0.2682926829268293, 'f': 0.24175823680714903}}]


In [ ]:
# Testing out pegasus model entire text

pegasus_candidate2_results = calc_rouge_scores(pegasus_candidate2, reference)
print(pegasus_candidate2_results)

[{'rouge-1': {'r': 0.24, 'p': 0.3076923076923077, 'f': 0.2696629164246939}, 'rouge-2': {'r': 0.07246376811594203, 'p': 0.1111111111111111, 'f': 0.08771929346722095}, 'rouge-l': {'r': 0.22, 'p': 0.28205128205128205, 'f': 0.2471910063123344}}]


unsurprisingly, the pegasus model, when using the entire text, produced not very good results; nowhere close to the baseline model

In [ ]:
# Testing out pegasus model by paragraphs

pegasus_candidate3_results = calc_rouge_scores(pegasus_candidate3, reference)
print(pegasus_candidate3_results)

[{'rouge-1': {'r': 0.34, 'p': 0.3469387755102041, 'f': 0.34343433843485366}, 'rouge-2': {'r': 0.14492753623188406, 'p': 0.17543859649122806, 'f': 0.1587301537755104}, 'rouge-l': {'r': 0.3, 'p': 0.30612244897959184, 'f': 0.30303029803081327}}]


but when fed using paragraphs, it looks swell! or does it? idk goodnight everybody

Let's now try on the sshleifer/distilbart-cnn-12-6 model

In [ ]:
sshleifer_results = calc_rouge_scores(sshleifer_candidate, reference)
print(sshleifer_results)

[{'rouge-1': {'r': 0.14, 'p': 0.14583333333333334, 'f': 0.1428571378592255}, 'rouge-2': {'r': 0.014492753623188406, 'p': 0.018518518518518517, 'f': 0.016260157675988006}, 'rouge-l': {'r': 0.14, 'p': 0.14583333333333334, 'f': 0.1428571378592255}}]


# Now let's try to fine-tune the pegasus model

In [ ]:
!pip install transformers datasets
!pip install sentencepiece
!pip install protobuf==3.20.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
googleapis-common-protos 1.58.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments
import torch

In [ ]:
class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels['input_ids'])  # len(self.labels)


In [ ]:
def prepare_data(model_name, 
                 train_texts, train_labels, 
                 val_texts=None, val_labels=None, 
                 test_texts=None, test_labels=None):
  """
  Prepare input data for model fine-tuning
  """
  tokenizer = PegasusTokenizer.from_pretrained(model_name)

  prepare_val = False if val_texts is None or val_labels is None else True
  prepare_test = False if test_texts is None or test_labels is None else True
  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True)
    decodings = tokenizer(labels, truncation=True, padding=True)
    dataset_tokenized = PegasusDataset(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  val_dataset = tokenize_data(val_texts, val_labels) if prepare_val else None
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, val_dataset, test_dataset, tokenizer

In [ ]:
def prepare_fine_tuning(model_name, tokenizer, train_dataset, val_dataset=None, freeze_encoder=False, output_dir='./results'):
  """
  Prepare configurations and base model for fine-tuning
  """
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

  if freeze_encoder:
    for param in model.model.encoder.parameters():
      param.requires_grad = False

  if val_dataset is not None:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=25,           # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      per_device_eval_batch_size=1,    # batch size for evaluation, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      evaluation_strategy='steps',     # evaluation strategy to adopt during training
      eval_steps=100,                  # number of update steps before evaluation
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset,            # evaluation dataset
      tokenizer=tokenizer
    )

  else:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=25,           # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      tokenizer=tokenizer
    )

  trainer.train()

In [ ]:
# using xsum for training
from datasets import load_dataset
dataset = load_dataset("xsum")
train_texts, train_labels = dataset['train']['document'][:100], dataset['train']['summary'][:100]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

KeyboardInterrupt: ignored

In [ ]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
# model.to(device)
# model.train()

# train_loader = DataLoader(dataset['train']['document'][:100], batch_size=16, shuffle=True)

# optim = AdamW(model.parameters(), lr=5e-5)

# for epoch in range(3):
#     for batch in train_loader:
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

# model.eval()

In [ ]:
# using peg fin sum model for fine-tuning
model_name = "human-centered-summarization/financial-summarization-pegasus"
train_dataset, _, _, tokenizer = prepare_data(model_name, train_texts, train_labels)
trainer = prepare_fine_tuning(model_name, tokenizer, train_dataset)
trainer.train()

In [ ]:
# run rogue score on trained model and see if model is better than before:


In [ ]:
# save the model
